In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import matplotlib.mlab as mlabPCA
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

life_df = pd.read_sql_query(
    'select * from lifeexpectancy',con=engine)
engine.dispose()

cols = life_df.columns

numeric_cols = [x for x in cols if x not in ('Country','Year','Status')]

life_var = []
print(life_df.info())
for i in life_df:
    life_var.append(i)

There are 22 variables
There are 64636 datapoints
Categorical variables: Country, Year, Status, Schooling
Continous variables: Life expectancy, Adult Mortality, Infant Deaths, Alchohol, Percentage expenditure, Hepatitus B, Measles, Under five deaths, Polio, Total expenditure, Diptheria, HIV/AIDS, GDP, Population, Thinness 1-19 years, thinness 5-9 years, income composition of resources, BMI

In [ ]:
for i in life_df:
    if i != 'Country' and i != 'Status':
        life_df[i].fillna(life_df[i].mean(), inplace=True)
        life_df[i] = winsorize(life_df[i], (0, 0.10))

Data missing:
There weren't that many missing values, so instead of dropping the entries, I filled them with the mean
I winsorized the values with high outliers, since I wanted to make sure the outliers didin't have an outsized impact on my data

Univariate Visualization:
Histograms of all variables except for country

In [ ]:
for i in life_df:
    if i != 'Country':
        plt.hist(life_df[i])
        plt.title("hist of {}".format(i))
        plt.show()

Probability distributions:
Normal distributions - total expenditure

In [ ]:
print(life_df.describe())

for i in life_df:
    if i != 'Country' and i != 'Status'and i != 'Year':
        print(i)
        print(life_df[i].mean())
        print(life_df[i].median())

        print(life_df[i].mode())

above, you can access the central tendencies of each variable

for i in life_df:
    if i != 'Country' and i != 'Status'and i != 'Year':
        print(i)
        print(life_df[i].var())

above, you can access the variance in each variable

Bivariate analysis:

In [ ]:
sns.heatmap(life_df.corr(), square=True, annot=True, linewidths=.5)
plt.title("correlation matrix")
plt.show()

The highest correlations with life expectancy seem to be Schooling, Income composition of resources, and BMI

Bivariate vizualization

In [ ]:
g = sns.lmplot(y='Life expectancy ', x='Schooling', data=life_df, fit_reg=True,
               scatter_kws={'alpha':0.4})
g.set_ylabels("Life expectancy")
g.set_xlabels("Schooling")
plt.title('Scatterplot with regression line: Life expectancy by schooling')
plt.show()

g = sns.lmplot(y='Life expectancy ',
                x='Income composition of resources',
                data=life_df, fit_reg=True,
               scatter_kws={'alpha':0.4})
g.set_ylabels("Life expectancy")
g.set_xlabels("Income composition of resources")
plt.title(
    'Scatterplot with regression line: Life expectancy by Income composition of resources')
plt.show()

g = sns.lmplot(y='Life expectancy ',
                x=life_var[10],
                data=life_df, fit_reg=True,
               scatter_kws={'alpha':0.4})
g.set_ylabels("Life expectancy")
g.set_xlabels("BMI")
plt.title('Scatterplot with regression line: Life expectancy by BMI')
plt.show()

After looking at the multivariate analysis, as well as the correlations, I would remove thinness 1-19 years and thinness 5-9 years, as they are highly correlated to
eachother, but not correlated to our main variable, life expectancy. I would do the same for under five deaths and infant deaths. Diptheria and Polio are highly correlated to eachother, but they are also. fairly correlated to life expectancy, so they will not be removed. 

Summary:
Life expectancy is strongly correlated to schooling, income composition of resources, and BMI. This makes sense as people with more schooling may know the proper channels for receiving care, people with more income will be able to afford care, and people with higher BMIs are more likely to have more income to spend on food and health services. Thinness from ages 1-19 are not good indicaters of BMI, as children tend to be smaller, and there is a smaller window in terms of BMI score (for example, An adult might be of proper weight
from 160 to 200 while a child may be of appropriate weight from 60 to 80.)